In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

## 実質GDP
### データの読み込み

In [2]:
df = pd.read_csv("gaku-jk2312.csv", encoding="shift-jis")
df = df.dropna()
df.head()

,四半期,国内総生産(支出側),民間最終消費支出,家計最終消費支出,除く持ち家の帰属家賃,民間住宅,民間企業設備,民間在庫変動,政府最終消費支出,公的固定資本形成,...,国内総所得,純受取,受取,支払,国民総所得,国内需要,民間需要,公的需要,総固定資本形成,最終需要
0,1994/ 1- 3.,446275.0,247502.0,243619.0,208574.0,29812.0,66259.0,4229,71357.0,47098.0,...,465653.0,4121.0,14527.0,10406.0,469774.0,462308.0,346440.0,115942.0,139766.0,442221.0
1,4- 6.,443830.0,248851.0,244912.0,209663.0,31119.0,66065.0,(2885),72244.0,48012.0,...,462572.0,3800.0,14474.0,10674.0,466372.0,460096.0,341695.0,118659.0,141500.0,445626.0
2,7- 9.,448931.0,250606.0,246645.0,211181.0,33628.0,65999.0,(281),72789.0,46294.0,...,467074.0,3871.0,14799.0,10927.0,470945.0,465711.0,348110.0,117728.0,142081.0,448128.0
3,10-12.,447124.0,250734.0,246728.0,211087.0,32218.0,67029.0,(1602),72950.0,45580.0,...,465653.0,3903.0,15316.0,11413.0,469556.0,463959.0,346832.0,117252.0,141344.0,447535.0
4,1995/ 1- 3.,452093.0,252909.0,248836.0,212992.0,31122.0,68648.0,2254,74470.0,43634.0,...,471048.0,3954.0,15604.0,11650.0,475002.0,469807.0,353586.0,116207.0,140426.0,449515.0


### 各変数名の確認

In [3]:
df.columns

Index(['四半期', '国内総生産(支出側)', '民間最終消費支出', '家計最終消費支出', '除く持ち家の帰属家賃', '民間住宅',
       '民間企業設備', '民間在庫変動', '政府最終消費支出', '公的固定資本形成', '公的在庫変動', '純輸出', '輸出', '輸入',
       '開差', '交易利得', '国内総所得', '純受取', '受取', '支払', '国民総所得', '国内需要', '民間需要',
       '公的需要', '総固定資本形成', '最終需要'],
      dtype='object')

### 必要な項目を抽出
- Y=C+G+I+(X-M)

に合致する項目を抽出

In [4]:
df_ycigxm = df[["国内総生産(支出側)", 
                "民間最終消費支出",
                "民間企業設備", 
                "政府最終消費支出", 
                "輸出", 
                "輸入"
              ]]
df_ycigxm.head()

,国内総生産(支出側),民間最終消費支出,民間企業設備,政府最終消費支出,輸出,輸入
0,446275.0,247502.0,66259.0,71357.0,37756.0,45476.0
1,443830.0,248851.0,66065.0,72244.0,37964.0,46328.0
2,448931.0,250606.0,65999.0,72789.0,38658.0,47726.0
3,447124.0,250734.0,67029.0,72950.0,38821.0,48199.0
4,452093.0,252909.0,68648.0,74470.0,39312.0,49812.0


### 説明変数と目的変数を分ける

In [5]:
y = df_ycigxm["国内総生産(支出側)"]
x = df_ycigxm.drop("国内総生産(支出側)", axis=1)

### 説明変数間の相関行列を出力

In [6]:
x.corr()

,民間最終消費支出,民間企業設備,政府最終消費支出,輸出,輸入
民間最終消費支出,1.000000,0.719911,0.858213,0.907647,0.891695
民間企業設備,0.719911,1.000000,0.747032,0.816505,0.845863
政府最終消費支出,0.858213,0.747032,1.000000,0.946206,0.964487
輸出,0.907647,0.816505,0.946206,1.000000,0.976244
輸入,0.891695,0.845863,0.964487,0.976244,1.000000


この結果から各項目において非常に強い相関が見られる。
### 相関行列の逆行列を作成
この逆行列から後にVIFを算出する。

In [7]:
inv = np.linalg.inv(x.corr().values)
inv

array([[  5.87459678,   0.87978464,   1.32602871,  -4.6039191 ,
         -2.76691566],
       [  0.87978464,   4.75684396,   4.72815934,  -0.22377797,
         -9.1499226 ],
       [  1.32602871,   4.72815934,  19.16353456,  -2.00658225,
        -21.70584605],
       [ -4.6039191 ,  -0.22377797,  -2.00658225,  25.09573409,
        -18.26965997],
       [ -2.76691566,  -9.1499226 , -21.70584605, -18.26965997,
         49.97746928]])

### VIFを算出
VIFは10を超えると多重共線性が認められる。ここで、多重共線性が見られると、ある説明変数の増減が起因して他の説明変数も増減する。

In [8]:
vif = []
for i in range(len(inv)):
    vif.append(inv[i][i])
df_vifj = pd.DataFrame(vif)
df_vifj.index = x.corr().columns
df_vifj.columns = ["VIF(実質GDP)"]
df_vifj

,VIF(実質GDP)
民間最終消費支出,5.874597
民間企業設備,4.756844
政府最終消費支出,19.163535
輸出,25.095734
輸入,49.977469


以上のVIF算出結果から
- 政府最終消費支出
- 輸出
- 輸入

において多重共線性が確認できた。
### 疑似相関の確認
相関行列を先ほど出力したが、相関には見かけ上の相関が存在し、第三の因子を交えることで相関係数が大きく変動する事がある。そのための計算が偏相関係数といい、そこで偏相関係数と相関係数の差の絶対値が大きい時に疑似相関が確認できる。

In [9]:
def partcorr(r_xy, r_yz, r_xz):
    return (r_xy - r_yz * r_xz) / (np.sqrt(1 - r_yz ** 2) * np.sqrt(1 - r_xz ** 2))

In [10]:
corr = x.corr().values
col = x.corr().columns
pcorr = []
for i in range(len(corr)):
    for j in range(i+1, len(corr[i])):
        for k in range(len(corr)):
            if col[i] != col[k] and col[j] != col[k] and col[i] and col[j]:
                pcorr.append([col[i], col[j], col[k], corr[i][j], 
                              partcorr(corr[i][j], corr[i][k], corr[k][j]), 
                              abs(corr[i][j]-partcorr(corr[i][j], corr[i][k], corr[k][j]))])
df_pcorr = pd.DataFrame(pcorr)
df_pcorr.columns = ["因子1", "因子2", "第三の因子", "相関係数", "偏相関係数", "相関係数と偏相関係数の差"]
df_pcorr.sort_values("相関係数と偏相関係数の差", ascending=False)

,因子1,因子2,第三の因子,相関係数,偏相関係数,相関係数と偏相関係数の差
14,民間企業設備,政府最終消費支出,輸入,0.747032,-0.488270,1.235302
17,民間企業設備,輸出,輸入,0.816505,-0.080151,0.896657
13,民間企業設備,政府最終消費支出,輸出,0.747032,-0.136773,0.883805
5,民間最終消費支出,政府最終消費支出,輸入,0.858213,-0.015185,0.873398
23,政府最終消費支出,輸出,輸入,0.946206,0.080932,0.865274
4,民間最終消費支出,政府最終消費支出,輸出,0.858213,-0.004479,0.862691
2,民間最終消費支出,民間企業設備,輸入,0.719911,-0.142237,0.862148
11,民間最終消費支出,輸入,輸出,0.891695,0.061691,0.830004
1,民間最終消費支出,民間企業設備,輸出,0.719911,-0.087433,0.807344
8,民間最終消費支出,輸出,輸入,0.907647,0.378637,0.529010


#### 政府最終消費支出の偏相関係数を確認する

In [11]:
df_pcorr.query("因子2=='政府最終消費支出' or 因子1=='政府最終消費支出'").sort_values("相関係数と偏相関係数の差", ascending=False)

,因子1,因子2,第三の因子,相関係数,偏相関係数,相関係数と偏相関係数の差
14,民間企業設備,政府最終消費支出,輸入,0.747032,-0.488270,1.235302
13,民間企業設備,政府最終消費支出,輸出,0.747032,-0.136773,0.883805
5,民間最終消費支出,政府最終消費支出,輸入,0.858213,-0.015185,0.873398
23,政府最終消費支出,輸出,輸入,0.946206,0.080932,0.865274
4,民間最終消費支出,政府最終消費支出,輸出,0.858213,-0.004479,0.862691
12,民間企業設備,政府最終消費支出,民間最終消費支出,0.747032,0.362644,0.384388
26,政府最終消費支出,輸入,輸出,0.964487,0.581368,0.383118
21,政府最終消費支出,輸出,民間最終消費支出,0.946206,0.776301,0.169905
3,民間最終消費支出,政府最終消費支出,民間企業設備,0.858213,0.694432,0.163780
24,政府最終消費支出,輸入,民間最終消費支出,0.964487,0.857477,0.107010


この結果から民間企業設備と政府最終消費支出は輸入を考慮すると負の相関に変化する。
### 重回帰分析を用いたGDPの式
ここで
- Y=C+G+I+(X-M)

の妥当性について検証する。ここで、与式には切片が無いためconstを作成しない。

In [12]:
model = sm.OLS(y, x).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:             国内総生産(支出側)   R-squared (uncentered):                   1.000
Model:                            OLS   Adj. R-squared (uncentered):              1.000
Method:                 Least Squares   F-statistic:                          3.088e+05
Date:                Sat, 08 Jul 2023   Prob (F-statistic):                   5.08e-230
Time:                        21:28:23   Log-Likelihood:                         -1146.2
No. Observations:                 117   AIC:                                      2302.
Df Residuals:                     112   BIC:                                      2316.
Df Model:                           5                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
民間最終消費支出       1.0744      0.036     30.104      0.000       1.004       1.145
民間企業設備         2.0241      0.104     19.478      0.000       1.818       2.230
政府最終消費支出       0.8524      0.117      7.290      0.000       0.621       1.084
輸出             0.2924      0.083      3.503      0.001       0.127       0.458
輸入            -0.7814      0.146     -5.367      0.000      -1.070      -0.493
==============================================================================
Omnibus:                        7.860   Durbin-Watson:                   0.624
Prob(Omnibus):                  0.020   Jarque-Bera (JB):               10.436
Skew:                          -0.344   Prob(JB):                      0.00542
Kurtosis:                       4.292   Cond. No.                         150.
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

この結果から本来の数式は
- Y=1.07C+0.852G+2.024I+(0.292X-0.781M)

となる。

### 影響度の検証
重回帰分析はp値と、標準化した各説明変数を用いることで係数から影響度を表すことができる。

#### 標準化

In [13]:
xt = x.copy()
for col in x.columns:
    xt[col] = (xt[col] - xt[col].mean()) / xt[col].std()
xt.describe()

,民間最終消費支出,民間企業設備,政府最終消費支出,輸出,輸入
count,1.170000e+02,1.170000e+02,1.170000e+02,1.170000e+02,1.170000e+02
mean,-1.397742e-15,4.915346e-16,2.467162e-16,3.605853e-17,2.827748e-16
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-2.266417e+00,-2.174233e+00,-1.888112e+00,-1.610603e+00,-1.890759e+00
25%,-7.528486e-01,-7.995070e-01,-6.725573e-01,-1.023879e+00,-8.827206e-01
50%,2.648064e-01,-1.299234e-01,-1.256434e-01,2.019413e-01,5.796565e-02
75%,7.655702e-01,9.002393e-01,8.536362e-01,8.509205e-01,9.657330e-01
max,1.706054e+00,1.856640e+00,1.855076e+00,1.640735e+00,1.895764e+00


この結果から平均値は0となり標準偏差は1になった。
#### 影響度の測定

In [14]:
xt = sm.add_constant(xt)
model = sm.OLS(y, xt).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             国内総生産(支出側)   R-squared:                       0.992
Model:                            OLS   Adj. R-squared:                  0.991
Method:                 Least Squares   F-statistic:                     2601.
Date:                Sat, 08 Jul 2023   Prob (F-statistic):          3.01e-113
Time:                        21:28:23   Log-Likelihood:                -1095.0
No. Observations:                 117   AIC:                             2202.
Df Residuals:                     111   BIC:                             2219.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       5.095e+05    266.496   1911.718      0.000    5.09e+05     5.1e+05
民間最終消費支出     1.04e+04    648.699     16.031      0.000    9113.531    1.17e+04
民間企業設備      9282.0720    583.732     15.901      0.000    8125.368    1.04e+04
政府最終消費支出    2781.0119   1171.634      2.374      0.019     459.341    5102.683
輸出          1.334e+04   1340.769      9.952      0.000    1.07e+04     1.6e+04
輸入         -3202.1104   1892.088     -1.692      0.093   -6951.409     547.188
==============================================================================
Omnibus:                       20.060   Durbin-Watson:                   0.995
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               30.245
Skew:                          -0.824   Prob(JB):                     2.71e-07
Kurtosis:                       4.868   Cond. No.                         17.5
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

この結果から実質GDPに影響を与えやすい変数は民間最終消費支出(C)と輸出(X)と民間企業設備(I)であることが分かる。逆に実質GDPにおいて政府最終消費支出は影響力が他の変数と比べて低いことが分かる。
## 名目GDP
### データの読み込み

In [15]:
df = pd.read_csv("gaku-mk2312.csv", encoding="shift-jis")
df = df.dropna()
df.head()

,四半期,国内総生産(支出側),民間最終消費支出,家計最終消費支出,除く持ち家の帰属家賃,民間住宅,民間企業設備,民間在庫変動,政府最終消費支出,公的固定資本形成,...,輸入,純受取,受取,支払,国民総所得,国内需要,民間需要,公的需要,総固定資本形成,最終需要
0,1994/ 1- 3.,510129.0,266613.0,262049.0,227606.0,28674.0,80433.0,3822,73547.0,46403.0,...,33933.0,4449.0,15710.0,11261.0,514578.0,499075.0,379542.0,119533.0,155511.0,506725.0
1,4- 6.,509013.0,269151.0,264515.0,229700.0,29970.0,79980.0,(3045),75113.0,47380.0,...,35524.0,4148.0,15650.0,11502.0,513161.0,499037.0,376056.0,122982.0,157330.0,511570.0
2,7- 9.,512969.0,270651.0,265985.0,230762.0,32363.0,79431.0,(138),75898.0,45541.0,...,36487.0,4190.0,16018.0,11828.0,517159.0,504402.0,382306.0,122096.0,157334.0,512450.0
3,10-12.,511573.0,271237.0,266518.0,230814.0,30940.0,80299.0,(1605),76697.0,44714.0,...,36525.0,4208.0,16563.0,12355.0,515781.0,502988.0,380871.0,122117.0,155953.0,512471.0
4,1995/ 1- 3.,514942.0,272765.0,267977.0,231905.0,29884.0,82050.0,2065,77679.0,42840.0,...,37246.0,4262.0,16862.0,12600.0,519205.0,507135.0,386764.0,120371.0,154774.0,513025.0


### 各変数名の確認

In [16]:
df.columns

Index(['四半期', '国内総生産(支出側)', '民間最終消費支出', '家計最終消費支出', '除く持ち家の帰属家賃', '民間住宅',
       '民間企業設備', '民間在庫変動', '政府最終消費支出', '公的固定資本形成', '公的在庫変動', '純輸出', '輸出', '輸入',
       '純受取', '受取', '支払', '国民総所得', '国内需要', '民間需要', '公的需要', '総固定資本形成', '最終需要'],
      dtype='object')

### 必要な項目を抽出
- Y=C+G+I+(X-M)

に合致する項目を抽出

In [17]:
df_ycigxm = df[["国内総生産(支出側)", 
                "民間最終消費支出",
                "民間企業設備", 
                "政府最終消費支出", 
                "輸出", 
                "輸入"
              ]]
df_ycigxm.head()

,国内総生産(支出側),民間最終消費支出,民間企業設備,政府最終消費支出,輸出,輸入
0,510129.0,266613.0,80433.0,73547.0,44988.0,33933.0
1,509013.0,269151.0,79980.0,75113.0,45500.0,35524.0
2,512969.0,270651.0,79431.0,75898.0,45054.0,36487.0
3,511573.0,271237.0,80299.0,76697.0,45110.0,36525.0
4,514942.0,272765.0,82050.0,77679.0,45054.0,37246.0


### 説明変数と目的変数を分ける

In [18]:
y = df_ycigxm["国内総生産(支出側)"]
x = df_ycigxm.drop("国内総生産(支出側)", axis=1)

### 説明変数間の相関行列を出力

In [19]:
x.corr()

,民間最終消費支出,民間企業設備,政府最終消費支出,輸出,輸入
民間最終消費支出,1.000000,0.512830,0.810333,0.879074,0.855870
民間企業設備,0.512830,1.000000,0.283093,0.448139,0.401654
政府最終消費支出,0.810333,0.283093,1.000000,0.886844,0.898936
輸出,0.879074,0.448139,0.886844,1.000000,0.964391
輸入,0.855870,0.401654,0.898936,0.964391,1.000000


この結果から基本的には各項目において非常に強い相関が見られるが、輸出と輸入と民間最終消費支出は民間企業設備との相関について実質より弱くなっており、特に政府最終消費支出は民間企業設備との相関は小さい。
### 相関行列の逆行列を作成
この逆行列から後にVIFを算出する。

In [20]:
inv = np.linalg.inv(x.corr().values)
inv

array([[  5.05498771,  -0.92959844,  -1.17242164,  -2.73752796,
         -0.25905898],
       [ -0.92959844,   1.52760611,   0.90964133,  -0.8731547 ,
          0.20639961],
       [ -1.17242164,   0.90964133,   5.99634811,  -1.59823282,
         -3.21093452],
       [ -2.73752796,  -0.8731547 ,  -1.59823282,  17.61046056,
        -12.85297904],
       [ -0.25905898,   0.20639961,  -3.21093452, -12.85297904,
         16.42053866]])

### VIFを算出
VIFは10を超えると多重共線性が認められる。ここで、多重共線性が見られると、ある説明変数の増減が起因して他の説明変数も増減する。

In [21]:
vif = []
for i in range(len(inv)):
    vif.append(inv[i][i])
df_vif = pd.DataFrame(vif)
df_vif.columns = ["VIF(名目GDP)"]
df_vif.index = x.corr().columns
df_vif

,VIF(名目GDP)
民間最終消費支出,5.054988
民間企業設備,1.527606
政府最終消費支出,5.996348
輸出,17.610461
輸入,16.420539


以上のVIF算出結果から
- 輸出
- 輸入

において多重共線性が確認できた。
### 疑似相関の確認
相関行列を先ほど出力したが、相関には見かけ上の相関が存在し、第三の因子を交えることで相関係数が大きく変動する事がある。そのための計算が偏相関係数といい、そこで偏相関係数と相関係数の差の絶対値が大きい時に疑似相関が確認できる。

In [22]:
corr = x.corr().values
col = x.corr().columns
pcorr = []
for i in range(len(corr)):
    for j in range(i+1, len(corr[i])):
        for k in range(len(corr)):
            if col[i] != col[k] and col[j] != col[k] and col[i] and col[j]:
                pcorr.append([col[i], col[j], col[k], corr[i][j], 
                              partcorr(corr[i][j], corr[i][k], corr[k][j]), 
                              abs(corr[i][j]-partcorr(corr[i][j], corr[i][k], corr[k][j]))])
df_pcorr = pd.DataFrame(pcorr)
df_pcorr.columns = ["因子1", "因子2", "第三の因子", "相関係数", "偏相関係数", "相関係数と偏相関係数の差"]
df_pcorr.sort_values("相関係数と偏相関係数の差", ascending=False)

,因子1,因子2,第三の因子,相関係数,偏相関係数,相関係数と偏相関係数の差
11,民間最終消費支出,輸入,輸出,0.855870,0.064247,0.791623
23,政府最終消費支出,輸出,輸入,0.886844,0.171912,0.714932
4,民間最終消費支出,政府最終消費支出,輸出,0.810333,0.139523,0.670810
5,民間最終消費支出,政府最終消費支出,輸入,0.810333,0.180782,0.629551
13,民間企業設備,政府最終消費支出,輸出,0.283093,-0.276795,0.559888
12,民間企業設備,政府最終消費支出,民間最終消費支出,0.283093,-0.263333,0.546427
26,政府最終消費支出,輸入,輸出,0.898936,0.357356,0.541580
20,民間企業設備,輸入,輸出,0.401654,-0.129115,0.530768
8,民間最終消費支出,輸出,輸入,0.879074,0.392436,0.486638
18,民間企業設備,輸入,民間最終消費支出,0.401654,-0.083923,0.485577


In [23]:
df_pcorr.query("因子2=='政府最終消費支出' or 因子1=='政府最終消費支出' or 第三の因子=='政府最終消費支出'").sort_values("相関係数と偏相関係数の差", ascending=False)

,因子1,因子2,第三の因子,相関係数,偏相関係数,相関係数と偏相関係数の差
23,政府最終消費支出,輸出,輸入,0.886844,0.171912,0.714932
4,民間最終消費支出,政府最終消費支出,輸出,0.810333,0.139523,0.670810
5,民間最終消費支出,政府最終消費支出,輸入,0.810333,0.180782,0.629551
13,民間企業設備,政府最終消費支出,輸出,0.283093,-0.276795,0.559888
12,民間企業設備,政府最終消費支出,民間最終消費支出,0.283093,-0.263333,0.546427
26,政府最終消費支出,輸入,輸出,0.898936,0.357356,0.541580
14,民間企業設備,政府最終消費支出,輸入,0.283093,-0.194342,0.477435
10,民間最終消費支出,輸入,政府最終消費支出,0.855870,0.496424,0.359446
7,民間最終消費支出,輸出,政府最終消費支出,0.879074,0.592538,0.286535
21,政府最終消費支出,輸出,民間最終消費支出,0.886844,0.624730,0.262114


この結果から政府最終消費支出は実質の時と異なり相関係数と偏相関係数の差が大きい場合無相関に近くなる。
### 重回帰分析を用いたGDPの式
ここで
- Y=C+G+I+(X-M)

の妥当性について検証する。ここで、与式には切片が無いためconstを作成しない。

In [24]:
model = sm.OLS(y, x).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:             国内総生産(支出側)   R-squared (uncentered):                   1.000
Model:                            OLS   Adj. R-squared (uncentered):              1.000
Method:                 Least Squares   F-statistic:                          3.099e+05
Date:                Sat, 08 Jul 2023   Prob (F-statistic):                   4.11e-230
Time:                        21:28:23   Log-Likelihood:                         -1150.6
No. Observations:                 117   AIC:                                      2311.
Df Residuals:                     112   BIC:                                      2325.
Df Model:                           5                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
民間最終消費支出       0.9904      0.038     26.383      0.000       0.916       1.065
民間企業設備         2.3989      0.084     28.523      0.000       2.232       2.566
政府最終消費支出       0.7377      0.091      8.142      0.000       0.558       0.917
輸出             0.3491      0.084      4.176      0.000       0.183       0.515
輸入            -0.7999      0.066    -12.038      0.000      -0.932      -0.668
==============================================================================
Omnibus:                        2.608   Durbin-Watson:                   0.694
Prob(Omnibus):                  0.271   Jarque-Bera (JB):                2.158
Skew:                          -0.186   Prob(JB):                        0.340
Kurtosis:                       3.552   Cond. No.                         86.1
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

この結果から名目GDPの数式は
- Y=0.990C+0.738G+2.400I+(0.349X-0.800M)

であることが分かる。
### 影響度の検証
重回帰分析はp値と、標準化した各説明変数を用いることで係数から影響度を表すことができる。

#### 標準化

In [25]:
xt = x.copy()
for col in x.columns:
    if col != "const":
        xt[col] = (xt[col] - xt[col].mean()) / xt[col].std()
xt.describe()

,民間最終消費支出,民間企業設備,政府最終消費支出,輸出,輸入
count,1.170000e+02,1.170000e+02,1.170000e+02,1.170000e+02,1.170000e+02
mean,2.216650e-15,1.859861e-16,4.934325e-16,-2.277381e-17,1.366428e-16
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-2.678610e+00,-2.217305e+00,-2.058173e+00,-1.542857e+00,-1.540692e+00
25%,-5.530789e-01,-7.397469e-01,-5.316937e-01,-9.218131e-01,-9.118670e-01
50%,-1.609539e-01,2.689370e-01,-1.685637e-01,-4.077049e-02,7.731822e-02
75%,6.923040e-01,7.192025e-01,7.870233e-01,8.933553e-01,7.170792e-01
max,2.896347e+00,2.200896e+00,2.248117e+00,2.592706e+00,3.087770e+00


この結果から平均値は0となり標準偏差は1になった。
#### 影響度の測定

In [26]:
xt = sm.add_constant(xt)
model = sm.OLS(y, xt).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             国内総生産(支出側)   R-squared:                       0.948
Model:                            OLS   Adj. R-squared:                  0.946
Method:                 Least Squares   F-statistic:                     407.9
Date:                Sat, 08 Jul 2023   Prob (F-statistic):           1.08e-69
Time:                        21:28:23   Log-Likelihood:                -1140.3
No. Observations:                 117   AIC:                             2293.
Df Residuals:                     111   BIC:                             2309.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       5.307e+05    392.398   1352.396      0.000     5.3e+05    5.31e+05
民間最終消費支出    5354.1544    886.035      6.043      0.000    3598.416    7109.893
民間企業設備      1.519e+04    487.076     31.196      0.000    1.42e+04    1.62e+04
政府最終消費支出    8190.8825    965.016      8.488      0.000    6278.639    1.01e+04
輸出          9597.6115   1653.775      5.803      0.000    6320.546    1.29e+04
輸入         -1.969e+04   1596.926    -12.329      0.000   -2.29e+04   -1.65e+04
==============================================================================
Omnibus:                       11.091   Durbin-Watson:                   0.760
Prob(Omnibus):                  0.004   Jarque-Bera (JB):               11.647
Skew:                          -0.651   Prob(JB):                      0.00296
Kurtosis:                       3.834   Cond. No.                         10.8
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

この結果から名目GDPに影響を与えやすいと考えられる変数は民間企業設備(I)と輸入(M)であることが分かる。政府最終消費支出(G)は輸出(X)と同程度だが輸出より小さい。